# Shallow Neural Networks

We will implement a shallow neural network to classify digits ranging from 0 to 9. The dataset you'll use is quite famous, it's called 'MNIST' http://yann.lecun.com/exdb/mnist/.


You might find [this notebook](https://nbviewer.jupyter.org/github/marc-moreaux/Deep-Learning-classes/blob/master/notebooks/dataset_MNIST.ipynb) to be usefull.

In [1]:
# Download the dataset in this directory 
#! wget http://deeplearning.net/data/mnist/mnist.pkl.gz

In [2]:
import pickle, gzip, numpy, math
import numpy as np

# Load the dataset
f = gzip.open('./Files/mnist.pkl.gz', 'rb')
train_set, valid_set, test_set = pickle.load(f,encoding="latin1")
f.close()

def to_one_hot(y, n_classes=10): 
    _y = np.zeros((len(y), n_classes))
    _y[np.arange(len(y)), y] = 1
    return _y

X_train, y_train = train_set[0], train_set[1]
X_valid, y_valid = valid_set[0], valid_set[1]
X_test,  y_test  = test_set[0],  test_set[1]


# We need to transform the output y from a number of 0 to 9, to a vector of 10 boolean values
y_train = to_one_hot(y_train)
y_valid = to_one_hot(y_valid)
y_test = to_one_hot(y_test)

---
# Implement a 2 layers NN

We will build a 2 layers SNN 
    - Layer 1 has 20 neurons with a sigmoid activation
    - Layer 2 has 10 neurons with a softmax activation
    - Loss is Negative Log Likelihood (wich is also the cross entropy)
    

### Define Parameters

In [3]:
        
def softmax(Z):
    """Z is a vector eg. [1,2,3]
    return: the vector softmax(Z) eg. [.09, .24, .67]
    """
    return np.exp(Z) / np.exp(Z).sum(axis=0)
    
def sigmoid(x):
    return (1/(1+np.exp(-x)))

np.random.seed(0)
inputSize=28
neuronSize=20
classifierSize=10
testSize = 50000

W1, b1 = np.random.normal(size=(inputSize*inputSize, neuronSize)),  np.random.normal(size=neuronSize)
W2, b2 = np.random.normal(size=(neuronSize, classifierSize)), np.random.normal(size=classifierSize)



### Define Model

In [4]:
def Pred(X, W1, W2, b1, b2):
    """Explanations ...
    Arguments:
        X: An input image (as a vector)(shape is <784,1>)
    Returns : a vector ???
    
    We can say that Pk = P(k|X;W) ; k belongs to [0;9]
    Then P = softmax(W * transposed(X))
    Because of the hidden layers, we have :
    A1 = W1 * transpose(X) + b1 (b1 being a bias)
    A2 = sigmoid(A1)
    z = W2 * transpose(A2) + b2
    and P = softmax(z)
    
    Hence : P = softmax( W2 * transpose[ sigmoid(W1*transpose[X]) ] )   -> the change between () & [] was done to ease reading
    """
    A1 = np.dot(W1.T, X) + b1
    A2 = sigmoid(A1)
    Z = np.dot(W2.T,A2) + b2
    P = softmax(Z)
    return P   
  
def loss(P, Y):
    """Explanations : 
    Arguments:
        P: The prediction vector corresponding to an image (X^s)
        Y: The ground truth of an image
    Returns: a vector ???
    
    We want to calculate the prediction : argmax(W) P(Y|X;W) so Y knowing X and the weights.
    We develop that into argmax(W) P(0|X;W) , ... , argmax(W) P(9|X;W)
    Hence the prediction becomes argmax(W) Π(i=0;9) & Π(j=0;9) OF ( P[i,j] ^ Y[i,j] )
    We can apply the natural logarithm function to simplify this to :
    L(W) = argmax(W) Σ(i=0;9) & Σ(j=0;9) OF (-Y[i,j]*ln(P[i,j]))
    This is the loss function, we want to reduce it.
    """
    ##############################(- sum(Y * np.log(P)))
    YP = -1* np.matmul(Y,np.log(P+0.000000000000001).T)
    loss = np.sum(YP)
    return loss


### Define Derivatives

In [5]:
def dW1(X, Y, P, W2, W1, b1):
    """Explanations 
    W1 is a weight matrix applied for each pixel (784) to each neuron of the first layer (20)
    
    Returns: A vector which is the derivative of the loss with respect to W1
    
    We want to minimize the loss function, so let's calculate dL/dW1 and 
    decompose it by going from P to W1 on the schema done in class:
    
    dL/dW1 = dL/dP * dP/dz * dz/dA2 * dA2/dA1 * dA1/dW1 
    
    dL/dP = Σ(i=0;9) & Σ(j=0;9) OF (-Y[i,j]/P[i,j])
    
    dP/dz = dP[i]/dz[j] = d/dz[j] * ( exp(z[i])/Σ(k=0;9) OF exp(z[k]) )
        Σ(k=0;9) OF exp(z[k]) will be Σ
        If i = j
            d/dz[j] * P[i] = [exp(z[i]) * Σ * exp(z[j]) * exp(z[i])]/[Σ²]
                           = [exp(z[i])/Σ] - [exp(z[i])²/Σ²]
                           = softmax(z[i]) - softmax(z[i])² 
                           = P[i] (1 - P[j]) 
                (i=j, so we will have the second term according to j for the sake of future explanation)
        
        If i != j
            d/dz[j] * P[i] = [0 * Σ - exp(z[j]) * exp(z[i])]/[Σ²] 
                    We have 0 since we derive exp(z[i]) by z[j] and i is never equal to j
                           = -[exp(z[j])/Σ]*[exp(z[i])/Σ]
                           = -P[i]*P[j]
                           = P[i] (0 - P[j])
                           
        So dP/dZ = P[i](δ[i,j] - P[j]) ; δ = {1 if i=j ; 0 otherwise}
        So if i=j, dP/dz is positive (P - P², P among [0,1], P>P²) 
        If it i!=j, dP/dz is negative (-P[i]P[j], P>0)
        
    dP/dZ = P[i](δ[i,j] - P[j]) ; δ = {1 if i=j ; 0 otherwise}
        
    dZ/dA2 = W2
    
    dA2/dA1 = A2/(1 - A2)
    
    dA1/dW1 = X
    
    But we can simplify dL/dP * dP/dz :
    dL/dz[i] = dL/dP*dP/dz[i] = Σ(j=0;9) OF ( (-Y[i,j]/P[j])*P[i]*(δ[i,j] - P[j]) ) 
                                #basic aggregation of the previous calculations
                        
                              = P[i] * Σ(j=0;9) OF ( (-Y[i,j]/P[j])*(δ[i,j] - P[j]) ) 
                              #We extract P[i] because it does not depend on j
                              
                              = P[i] * Σ(j=0;9) OF ( ( (-Y[i,j]*δ[i,j]/P[j]) + Y[j]) 
                              #We distribute Y/P to (δ - P)
                              
                              = P[i] * [ Σ(j=0;9) OF (-Y[i,j]*δ[i,j]/P[j]) + Σ(j=0;9) OF (Y[j]) ] 
                              #we divide the sum in 2 parts
                              
                     We have  Σ(j=0;9) OF (-Y[j]*δ)/P[j]  which is equal to -Y[i]/P[i], i=j, because it is 0 if i!=j
                     And Σ(j=0;9) OF (Y[j]) is equal to 1 since Y is that type of vector : [0,1,0,0,0,0,0,0,0,0]
                              
                              = P[i] * [-Y[i]/P[i] + 1] = -Y[i] + P[i]
    dL/dz[i] = P[i] - Y[i]
                              
    So, we can finally deduce that
    
    dL/dW1 = (P[i] - Y[i]) *  W2  *    A2    *  (1-A2) *     X 
    SIZE :      [1 * 10]    [10*20]  [20 * 1]   [20 * 1]  [1 * 28²]
                [1 * 10]          [10 * 1]          [20 * 28²]
                        [1 * 1]              [20 * 28²]
      It is a vector of size  [20,28²].
      
N.B. : When calculating size, we overpass the transpose problem and simply verify the dimension.
        """
    
    A1 = np.dot(W1.T, X) + b1
    A2 = sigmoid(A1)
    
    dW1 = np.matrix(np.dot(np.dot((P - Y), W2.transpose()), (A2 * (1 - A2)).transpose()) * X).transpose()

    """PY = np.subtract(P,Y)

    A2_1_A2 = np.matmul(A2.T,(np.ones(A2.shape)-A2))

    
    PYW = np.matmul(PY.T,W2.T)
    
    PYW_A21A2 = np.matmul(PYW,A2_1_A2)
    
    dW1 = np.matmul(X,PYW_A21A2)"""

    
    return dW1

def db1(X, Y, P, W2, W1, b1):
    """Explanations 
    The b1 are the biases applied to the first layer of neuron (20,1)
    
    Arguments:
        L is the loss af a sample (a scalar)
    Returns: A scalar which is the derivative of the Loss with respect to b1
    
    dL/db1 = dL/dP * dP/dz * dz/dA2 * dA2/dA1 * dA1/db1
        That is basically equal to dL/dW1, except that the last term was dA1/dW1 = X, now it is dA1/db1 = 1
    So dL/dB1 = (P[i] - Y[i]) *  W2  *     A2 *    (1-A2)  , we don't need L as an argument though
    SIZE :        [1 * 10]    [10*20]  [20 * 1]   [20 * 1]
                        [1 * 20]            [20 * 20]
      It is a matrix of size [1,20] despite the comment before.
    """
    A1 = np.dot(W1.T, X) + b1
    A2 = sigmoid(A1)
    return np.dot(np.dot((P - Y), W2.transpose()), (A2 * (1 - A2)).transpose())


def dW2(X, Y, P, W1, b1):
    """Explanations 
    W2 is a weight matrix too, it is applied for each neuron from the first layer (20) to each neuron from the second layer (10)

    With the same logic, let's calculate dL/dW2 :
    
    dL/dW2 = dL/dP * dP/dz * dz/dW2
           = (P[i] - Y[i]) *   A2
    SIZE :    [10 * 1]      [1 * 20]
    dL/dW2 is a vector of size [10,20].
    """
    A1 = np.dot(W1.T, X) + b1
    A2 = sigmoid(A1)
    return (np.matrix(P - Y).transpose()*A2).transpose()

def db2(P, Y):
    """Explanations 
    The b2 are the biases applied to the first layer of neuron (10,1)
    
    Same logic,
    
    dL/db2 = dL/dP * dP/dz * dz/db2
           = (P[i]-Y[i]) * 1
           = P[i] - Y[i]
    So it is a vector of size [10,1]
    
    
    """
    db2 = np.subtract(P,Y)
    return db2


### Model training


In [6]:
m = X_train.shape[0] # number of samples
alpha = 0.1 # learning rate

nbIter = 50
# Optimization loop
"""With a training dataset as big as this one (50000 samples), the computation is extremely long.
   For this reason, we can only loop over 10 iterations so it doesn't take too long. (10 minutes)
   But the results aren't very good since the algorithm does not have enough iterations to optimize the parameters well enough.
"""
for i in range(nbIter):
    print("Iteration number ",i)

    # Make prediction for each sample with current model paramaters
    P = np.array([Pred(x, W1, W2, b1, b2) for x in X_train])

    # Compute the global dataset value for the derivative of the loss with respect to each parameter.
    # This is achieved by computing the average of the derivatives for all the samples.
    dW1_global = (1/m) * sum(np.array([dW1(X_train[i], y_train[i], P[i], W2, W1, b1) for i in range(m)]))
    db1_global = (1/m) * sum(np.array([db1(X_train[i], y_train[i], P[i], W2, W1, b1) for i in range(m)]))
    dW2_global = (1/m) * sum(np.array([dW2(X_train[i], y_train[i], P[i], W1, b1) for i in range(m)]))
    db2_global = (1/m) * sum(np.array([db2(y_train[i], P[i]) for i in range(m)]))

    # Update model parameters using standard gradient descent
    W1 = W1 - (alpha * dW1_global)
    b1 = b1 - (alpha * db1_global)
    W2 = W2 - (alpha * dW2_global)
    b2 = b2 - (alpha * db2_global)
    

Iteration number  0
Iteration number  1
Iteration number  2
Iteration number  3
Iteration number  4
Iteration number  5
Iteration number  6
Iteration number  7
Iteration number  8
Iteration number  9
Iteration number  10
Iteration number  11
Iteration number  12
Iteration number  13
Iteration number  14
Iteration number  15
Iteration number  16
Iteration number  17
Iteration number  18
Iteration number  19
Iteration number  20
Iteration number  21
Iteration number  22
Iteration number  23
Iteration number  24
Iteration number  25
Iteration number  26
Iteration number  27
Iteration number  28
Iteration number  29
Iteration number  30
Iteration number  31
Iteration number  32
Iteration number  33
Iteration number  34
Iteration number  35
Iteration number  36
Iteration number  37
Iteration number  38
Iteration number  39
Iteration number  40
Iteration number  41
Iteration number  42
Iteration number  43
Iteration number  44
Iteration number  45
Iteration number  46
Iteration number  47
It

### Test of the accuracy of the model on the Test set

In [7]:
""" Test of our first model (optimized using all training samples)
"""
m_test =  X_test.shape[0] # number of test samples

# Compute predictions of output for test set using computed optimized model parameters
P_test = np.array([Pred(x, W1, W2, b1, b2) for x in X_test])

# Compute loss of model parameters for each sample
loss_global = np.array([loss(P_test[i], y_test[i]) for i in range(m_test)])
# Compute total loss
loss_total = sum(loss_global)

# For better visualization of results,
# transform the outputs and predictions into actual numbers, using argmax (index of max probability)
y_test_argmax = np.array([np.argmax(y_test[i]) for i in range(m_test)])
P_test_argmax = np.array([np.argmax(P_test[i]) for i in range(m_test)])

print("Loss per sample: {}".format(loss_global))
print("Total loss: {}".format(loss_total))
print("\nComparison of actual and predicted output for first 25 test samples (for visualization):")
print("Actual output:     {}".format(y_test_argmax[:25]))
print("Predicted output:  {}".format(P_test_argmax[:25]))

Loss per sample: [ 3.59642753  6.56135011  0.65771282 ...,  3.52756302  3.82871969
  1.11072133]
Total loss: 33464.474734111915

Comparison of actual and predicted output for first 25 test samples (for visualization):
Actual output:     [7 2 1 0 4 1 4 9 5 9 0 6 9 0 1 5 9 7 3 4 9 6 6 5 4]
Predicted output:  [6 6 1 6 6 1 6 1 1 6 6 1 6 6 6 1 6 6 6 1 6 1 1 1 1]


## Optimisation

In [8]:
""" Since the computations took too long when we took into account all 50000 training samples, 
    we tried to optimize the parameters using only a sample of the training set, reducing its size from 50000 to 1000.
    This optimization took about 16 minutes.
"""
nbIter = 1500
batch_size = 1500

# Take only first 1000 samples from traing set
X_sample = X_train[:batch_size]
y_sample = y_train[:batch_size]

m_sample = X_sample.shape[0] #number of samples

#Initialize new parameters
W1_sample, b1_sample = np.random.normal(size=(inputSize*inputSize, neuronSize)),  np.random.normal(size=neuronSize)
W2_sample, b2_sample = np.random.normal(size=(neuronSize, classifierSize)), np.random.normal(size=classifierSize)

# Optimization loop
for i in range(nbIter):
    # Make prediction for each sample with current model paramaters
    P_sample = np.array([Pred(x, W1_sample, W2_sample, b1_sample, b2_sample) for x in X_sample])

    # Compute the global dataset value for the derivative of the loss with respect to each parameter.
    # This is achieved by computing the average of the derivatives for all the samples.
    dW1_global_sample = (1/m_sample) * sum(np.array([dW1(X_sample[i], y_sample[i], P_sample[i], W2_sample, W1_sample, b1_sample) for i in range(m_sample)]))
    db1_global_sample = (1/m_sample) * sum(np.array([db1(X_sample[i], y_sample[i], P_sample[i], W2_sample, W1_sample, b1_sample) for i in range(m_sample)]))
    dW2_global_sample = (1/m_sample) * sum(np.array([dW2(X_sample[i], y_sample[i], P_sample[i], W1_sample, b1_sample) for i in range(m_sample)]))
    db2_global_sample = (1/m_sample) * sum(np.array([db2(y_sample[i], P_sample[i]) for i in range(m_sample)]))

    # Update model parameters using standard gradient descent
    W1_sample = W1_sample - (alpha * dW1_global_sample)
    b_sample1 = b1_sample - (alpha * db1_global_sample)
    W2_sample = W2_sample - (alpha * dW2_global_sample)
    b2_sample = b2_sample - (alpha * db2_global_sample)

In [9]:
""" Test of our second model (optimized using only a sample of the training set)
"""

m_test =  X_test.shape[0] # number of test samples

# Compute predictions of output for test set using computed optimized model parameters
P_sample_test = np.array([Pred(x, W1_sample, W2_sample, b1_sample, b2_sample)for x in X_test])

# Compute loss of model parameters for each sample
loss_sample_global = np.array([loss(P_sample_test[i], y_test[i]) for i in range(m_test)])
# Compute total loss
loss_sample_total = sum(loss_sample_global)

# For better visualization of results,
# transform the outputs and predictions into actual numbers, using argmax (index of max probability)
y_test_argmax = np.array([np.argmax(y_test[i]) for i in range(m_test)])
P_sample_test_argmax = np.array([np.argmax(P_sample_test[i]) for i in range(m_test)])

print("Loss: {}".format(loss_sample_global))
print("Total loss: {}".format(loss_sample_total))
print("\nComparison of actual and predicted output for first 25 test samples (for visualization):")
print("Actual output:     {}".format(y_test_argmax[:25]))
print("Predicted output:  {}".format(P_sample_test_argmax[:25]))

Loss: [ 0.15080078  1.04671801  0.68424589 ...,  1.97371365  2.1148837
  0.64010547]
Total loss: 14799.092644300494

Comparison of actual and predicted output for first 25 test samples (for visualization):
Actual output:     [7 2 1 0 4 1 4 9 5 9 0 6 9 0 1 5 9 7 3 4 9 6 6 5 4]
Predicted output:  [7 2 1 0 4 1 9 3 4 7 0 1 7 0 9 5 8 7 2 5 7 8 9 4 4]
